# S1: Insertion-Only Dynamic Connectivity in General Disk Graphs\[1\]

*Author: Lars Nitzschke* 

This notebook is part of the work in the seminar **Seminar on Algorithms**. More precisely it implements the algorithms and data structure discussed in the paper **"Insertion-Only Dynamic Connectivity in General Disk Graphs"** by Kaplan et al.[1]
The notebook offers interactive visualisations and animations.

## Table of Contents

1. Introduction
1. Setup
1. Implementation
1. References  

## 1. Introduction

## 2. Setup

First let's do some setup. This is not very interesting, so you can skip to Section 3 if you want.

We now import everything we'll need throughout this notebook from external sources, including our module for generic data structures, our module for geometric primitives and operations as well as our module for visualisation purposes. 
These modules are explained in [notebook no. 00](./00-Basics.ipynb).

In [19]:
# Python standard library imports
from abc import ABC, abstractmethod
from typing import Any, Optional, Iterable
import random

# Data structure, geometry and visualisation module imports
from modules.data_structures import Comparator, ComparisonResult as CR, HalfEdge, DoublyConnectedEdgeList, PointLocation, Face, VDLineSegment, AnimationBinaryTreeDict, PLSearchStructure
from modules.geometry import Point, PointReference, PointSequence, LineSegment, EPSILON, Rectangle, VerticalOrientation as VORT, HorizontalOrientation as HORT
from modules.visualisation import VisualisationTool, PointLocationInstance, VerticalExtensionMode, PointLocationMode, DiskMode, DiskSetInstance, LineSegmentsMode
from modules.geometry import Disk
from modules.data_structures import DisjointSet, ComponentTree

Additionally, we create an object for our visualisation tool. <!-- and register a few example instances.-->

In [20]:
visualisation_tool = VisualisationTool(400, 400, DiskSetInstance())
canvas_size = min(visualisation_tool.width, visualisation_tool.height)

## 3. Implementation

In [ ]:
def naive_connectivity(disk_set: set[Disk]) -> PointSequence:
    point_seq = PointSequence()
    for disk in disk_set:
        for other_disk in disk_set:
            if disk == other_disk:
                continue
            if disk.intersects(other_disk):
                point_seq.append(disk.center_point)
                point_seq.append(other_disk.center_point)
    return point_seq

def disjoint_set_connectivity(disk_set: set[Disk]) -> PointSequence:
    point_seq = PointSequence()
    disjoint_set = DisjointSet[Disk]()
    for disk in disk_set:
        disjoint_set.make_set(disk)
    for disk in disk_set:
        for other_disk in disk_set:
            if disk == other_disk:
                continue
            if disk.intersects(other_disk):
                #if disjoint_set.find(disk) != disjoint_set.find(other_disk):
                    disjoint_set.union(disk, other_disk)
                    #point_seq.append(disk.center_point)
                    #point_seq.append(other_disk.center_point)
    return point_seq

def component_tree_connectivity(disk_set: set[Disk]) -> PointSequence:
    point_seq = PointSequence()
    component_tree = ComponentTree()
    disjoint_set = DisjointSet[Disk]()
    for disk in disk_set:
        disjoint_set.make_set(disk)
        connected_components = component_tree.query(disk)
        # merge all connected components
        for connected_component in connected_components:
            disjoint_set.union(disk, connected_component[0])   
        component_tree.insert_disk(disk)
        #component_tree.verify_tree()

    #for disk in disk_set:
    #    for other_disk in disk_set:
    #        if disk == other_disk:
    #            continue
    #        if disjoint_set.in_same_set(disk, other_disk):
    #            if disk.intersects(other_disk):
    #                point_seq.append(disk.center_point)
    #                point_seq.append(other_disk.center_point)

    return point_seq

                

visualisation_tool.register_algorithm("Naive Connectivity", naive_connectivity, LineSegmentsMode(vertex_radius = 0, highlight_radius = 0))
visualisation_tool.register_algorithm("Disjoint Set Connectivity", disjoint_set_connectivity, LineSegmentsMode(vertex_radius = 0, highlight_radius = 0))
visualisation_tool.register_algorithm("Component Tree Connectivity", component_tree_connectivity, LineSegmentsMode(vertex_radius = 0, highlight_radius = 0))
visualisation_tool.display()

In [22]:
visualisation_tool._algorithm_buttons[2].click()

In [23]:
visualisation_tool._random_button.click()

In [24]:
print(visualisation_tool._instance._instance.__len__())
print(visualisation_tool._instance._instance)

10
{((280.51771948607353, 330.6576058031348) -r- 40.0), ((22.098139686624663, 227.66610110464458) -r- 40.0), ((179.3188930231671, 192.48784100925172) -r- 40.0), ((73.35917709274933, 252.77956692180913) -r- 40.0), ((62.27067649523179, 256.0528486414196) -r- 39.99999999999999), ((101.03215204472679, 72.14330749873773) -r- 40.0), ((262.08340518500245, 301.2307999061042) -r- 40.0), ((282.93870576322144, 215.42733685523916) -r- 40.0), ((178.2645762391603, 112.44212574522037) -r- 40.0), ((45.03958935142349, 47.55373895410734) -r- 40.0)}


## 4. References

\[1\] Haim Kaplan and Katharina Klost and Kristin Knorr and Wolfgang Mulzer and Liam Roditty. **Insertion-Only Dynamic Connectivity in General Disk Graphs.**, in 2024 Symposium on Simplicity in Algorithms (SOSA), doi: 10.1137/1.9781611977936.27